In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.ar_model import AutoReg
from sklearn.neural_network import MLPRegressor

# Load the Mauna Loa CO2 dataset
data = pd.read_csv("co2_mm_mlo.csv")

# Preprocess the dataset for monthly readings
monthly_data = data[['average']]
monthly_data.index = pd.to_datetime(data[['year', 'month']].assign(day=1))

# Split the dataset into training and testing sets
train_size = int(len(monthly_data) * 0.8)
train_data, test_data = monthly_data[:train_size], monthly_data[train_size+1:]

In [16]:
# Define functions for MA, ARMA, and MLP models
def moving_average(train, test, k):
    predictions = []
    history = list(train[-k:])
    for t in range(len(test)):
        yhat = np.mean(history)
        predictions.append(yhat)
        history.append(test[t])
    return predictions

def arma(train, test, p, q):
    predictions = []
    history = list(train)
    for t in range(len(test)):
        model = ARIMA(history, order=(p, 0, q))
        model_fit = model.fit()
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
    return predictions

def mlp(train, test, k):
    X_train, y_train = [], []
    for i in range(len(train) - k):
        X_train.append(train[i:i+k])
        y_train.append(train[i+k])
    X_test = [test[i:i+k] for i in range(len(test) - k)]
    
    mlp_regressor = MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=1000)
    mlp_regressor.fit(X_train, y_train)
    predictions = mlp_regressor.predict(X_test)
    return predictions

In [17]:
from sklearn.model_selection import KFold
def find_best_mlp_params(train_values, k_folds=5):
    param_grid = {
        'K': [3, 6, 9, 12],
        'T': [1, 2, 3]
    }

    best_rmse = float('inf')
    best_params = {}

    kf = KFold(n_splits=k_folds)

    for k_param in param_grid['K']:
        for t_param in param_grid['T']:
            rmse_sum = 0
            for train_index, val_index in kf.split(train_values):
                X_train_fold, X_val_fold = train_values[train_index], train_values[val_index]
                mlp_forecasts = mlp(X_train_fold, X_val_fold, k_param)
                rmse_fold = np.sqrt(mean_squared_error(X_val_fold[:len(mlp_forecasts)], mlp_forecasts))
                rmse_sum += rmse_fold
            avg_rmse = rmse_sum / k_folds
            if avg_rmse < best_rmse:
                best_rmse = avg_rmse
                best_params['K'] = k_param
                best_params['T'] = t_param

    return best_params

train_values = train_data['average'].values
test_values = test_data['average'].values

# Find the best parameters for MLP using k-fold cross-validation
best_params = find_best_mlp_params(train_values)
print("Best parameters for MLP:", best_params)

# Generate forecasts using the best parameters
mlp_forecasts = mlp(train_values, test_values, best_params['K'])

# Evaluate forecasts using RMSE
mlp_rmse = np.sqrt(mean_squared_error(test_values[:len(mlp_forecasts)], mlp_forecasts))

print("RMSE for Multi-Layer Perceptron (MLP):", mlp_rmse)

Best parameters for MLP: {'K': 3, 'T': 3}
RMSE for Multi-Layer Perceptron (MLP): 1.6480238231604816


In [18]:
# Define parameters
K = best_params['K']  
T = best_params['T']  

# Prepare data for modeling
train_values = train_data['average'].values
test_values = test_data['average'].values

# Generate forecasts
ma_forecasts = moving_average(train_values, test_values, K)
arma_forecasts = arma(train_values, test_values, 1, 1)  # ARMA(1,1)
mlp_forecasts = mlp(train_values, test_values, K)

c:\Users\Asus\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\Asus\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'


In [19]:
# Evaluate forecasts using RMSE
ma_rmse = np.sqrt(mean_squared_error(test_values[:len(ma_forecasts)], ma_forecasts))
arma_rmse = np.sqrt(mean_squared_error(test_values[:len(arma_forecasts)], arma_forecasts))
mlp_rmse = np.sqrt(mean_squared_error(test_values[:len(mlp_forecasts)], mlp_forecasts))

print("RMSE for Moving Average (MA):", ma_rmse)
print("RMSE for AutoRegressive Moving Average (ARMA):", arma_rmse)
print("RMSE for Multi-Layer Perceptron (MLP):", mlp_rmse)

RMSE for Moving Average (MA): 9.934895856546449
RMSE for AutoRegressive Moving Average (ARMA): 1.0479758441319134
RMSE for Multi-Layer Perceptron (MLP): 1.2532738696720505


#### Moving Average (MA) Model: 
    - The MA model typically has a high RMSE compared to more sophisticated models like ARMA and MLP. This is because the MA model only considers the historical average and does not account for any underlying trends or seasonality in the data.

#### AutoRegressive Moving Average (ARMA) Model:
    - The ARMA model shows the lowest RMSE among the three models, indicating the best forecasting performance in this particular analysis. Despite the warning messages regarding parameter initialization, the ARMA model demonstrates effectiveness in capturing the underlying patterns in the data.

#### Multi-Layer Perceptron (MLP) Model: 
    - The MLP model achieves a higher RMSE compared to the ARMA model, suggesting slightly lower forecasting accuracy. While the MLP model typically has the capacity to capture complex patterns, in this instance, it appears that the ARMA model outperforms it.

Therefore, in this specific analysis, the AutoRegressive Moving Average (ARMA) model appears to be the most effective for forecasting CO2 levels using the Mauna Loa dataset, followed by the Multi-Layer Perceptron (MLP) model.